# Movie Tool

In [ ]:
%pip install requests
%pip install pandas

In [ ]:
import requests
import pandas as pd
import numpy as np

In [ ]:
# the cookie for making API request to Netflix
# it can be retrieved from Chrome Dev Tool, after logging to Netflix
cookie = 'memclid=b783e450-fa77-488c-b3e2-beff624d26da; nfvdid=BQFmAAEBEEVEA6OC5bEMUUnOPrODS8pgtswkpg-fPl9nVBvzgBL6h4xcvm1AW25uw7kCk3ontTzlyk_IFIznVD7tQB97u8Sb9yS3ZTnKWCiKr7ML5MnwT0FDrZ5vU5D__d-e0TaPHF35IjWRoAUBT5CD29YZRJBC; netflix-sans-normal-3-loaded=true; netflix-sans-bold-3-loaded=true; SecureNetflixId=v%3D2%26mac%3DAQEAEQABABSAzFtR1_DQDeTeE_cs6BSVZTnOPPH7nhw.%26dt%3D1678850528044; NetflixId=ct%3DBQAOAAEBEPZtIjiITaLAjPHmT8jTaq6CUJGFXdchN_Zw-Vx7cAp_OLQ4n1G40FIEEK7V31GV7ekZfVgrcYKqn5MiwcSe29CMiHtsofe2gCh82PpPrA73UhReC-q_dzk5uevCmNNqO0WSa9kZ43iv1s3gOBSLRVVb-DGNKlwycF5hjDc35qDaHwu_CFr3_IGHxJv3penntkSncxthYn-7nPjP55hxpY3igY92-RliOVFx_AByre7Y67bbFX_HqKUMHM9korEcHEqNFGmvMwzynjNidRg4AcR_amRe6L76e9EwFQOObonBiBg5GUe4N0nNZtk5UPCW66cbQaUqdiKQLZuym2IXqeddHQXJEnhia4E-8yV-b67xHWt954Whefxdup61cVwvNsByhcbNXi3NGrbNJYkJtCVfd082MLjjTEFtvWAXUa3q9k17iab3NEE_awvaqhjeHSI2ZllJkH0GC57LZ4S5ICXLXnmpvNbjFjaDXAWheR7rFViVBTg-MlJZMRtmaWfa8N-Ho6_DP7a6_vqjtz6LYkVAtrju_h66QGuUDmfSbXGsfF3T9OcDW1ei2mvKIh6HQG2LzGvGR8yFSUXLKvth5C3HTph9GnxNVUW0cad136VQALpw81bxbdH_bd3xjL0EXSRNAJMJSywgllhFefBzEjCwV42maxqdVFi6AUDi1uJPw4Y6mLn4P-AwXxveScx8feUM9qSy5jlZUMTdVVwgsKNarvLVzgaUZF0q9NrkmQPvP1ZQNE20c8eOBgOuJYA8_Yx8zC0PKvvcXC1tKHrP9Ep16LqNbJte6cIAjWRDuV7nFgY.%26bt%3Ddbl%26ch%3DAQEAEAABABQnw__eUldKi0gl_1tUcwDpV3QssXhziJU.%26v%3D2%26mac%3DAQEAEAABABRdvju3TEOdz_bObCmuQ6s-5JmLA592VCg.; profilesNewSession=0; OptanonConsent=isIABGlobal=false&datestamp=Wed+Mar+15+2023+11%3A00%3A35+GMT%2B0700+(Indochina+Time)&version=202301.1.0&consentId=22b81ae4-c299-4a0c-ac36-055cb8f2e01c&interactionCount=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0002%3A1%2CC0003%3A1%2CC0004%3A1&hosts=&AwaitingReconsent=false&isGpcEnabled=0'

## Load movie list

In [ ]:
mov_file = 'data/movies.csv'
df = pd.read_csv(mov_file, index_col='Netflix Id')
len(df)

## Save movie list

In [ ]:
df.to_csv('data/movies.csv')

## Get My List movies

Fetch movies from My List

In [ ]:
def get_my_list():
  url = 'https://www.netflix.com/nq/website/memberapi/vc6b505ac/pathEvaluator?avif=false&webp=true&drmSystem=widevine&isVolatileBillboardsEnabled=true&routeAPIRequestsThroughFTL=false&isTop10Supported=true&isTop10KidsSupported=true&hasVideoMerchInBob=true&hasVideoMerchInJaw=true&persoInfoDensity=false&infoDensityToggle=false&contextAwareImages=true&enableMultiLanguageCatalog=false&usePreviewModal=true&falcor_server=0.1.0&withSize=true&materialize=true&original_path=%2Fshakti%2Fmre%2FpathEvaluator'
  maxItemIndex = 1000
  payload=f'path=%5B%22mylist%22%2C%5B%22id%22%2C%22listId%22%2C%22name%22%2C%22requestId%22%2C%22trackIds%22%5D%5D&path=%5B%22mylist%22%2C%7B%22from%22%3A0%2C%22to%22%3A{maxItemIndex}%7D%2C%5B%22availability%22%2C%22episodeCount%22%2C%22inRemindMeList%22%2C%22itemSummary%22%2C%22queue%22%2C%22summary%22%5D%5D&authURL=1656735386949.ilLXtrHw22MnWNrX3jwMASMQZwQ%3D'
  headers = {
    'content-type': 'application/x-www-form-urlencoded',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
    'Cookie': cookie
  }
  response = requests.request("POST", url, headers=headers, data=payload)

  if response.status_code != 200:
    raise Exception(f'Can not call API, status code: {response.status_code}, error: {response.text}')
  myListData = response.json()
  items = []
  for movId in myListData['jsonGraph']['videos']:
    # itemSummary / value {
    #     episodeCount, seasonCount, isOriginal, releaseYear
    #     boxArt / { url, width, height }
    #     maturity / rating / { value, specificRatingReason, maturityLevel }
    #   }
    video = myListData['jsonGraph']['videos'][movId]
    episode_cnt = video['itemSummary']['value']['seasonCount'] if 'seasonCount' in video['itemSummary']['value'] else None
    item = {
      'Netflix Id': video['itemSummary']['value']['id'],
      'Title': video['itemSummary']['value']['title'],
      'Release Year': video['itemSummary']['value']['releaseYear'],
      'Netflix Original': video['itemSummary']['value']['isOriginal'],
      'Episode Count': video['itemSummary']['value']['episodeCount'] if 'episodeCount' in video['itemSummary']['value'] else None,
      'Season Count': episode_cnt,
      'Maturity': video['itemSummary']['value']['maturity']['rating']['value'],
      'Maturity Level': video['itemSummary']['value']['maturity']['rating']['maturityLevel'],
      'Maturity Detail': video['itemSummary']['value']['maturity']['rating']['specificRatingReason'],
      'Image': video['itemSummary']['value']['boxArt']['url'],
      'Movie Type': 'Movie' if episode_cnt is None else 'TV Series'
    }
    items.append(item)
  return items

my_list_movies = get_my_list()
len(my_list_movies)

Add new movies to movie list

In [ ]:
new_movies = [mov for mov in my_list_movies if mov['Netflix Id'] not in df.index]
if len(new_movies) > 0:
  df = pd.concat([df, pd.DataFrame.from_records(new_movies, index='Netflix Id')])
len(df)

## Find Last day to watch on Netflix

In [ ]:
import re
import datetime

def get_last_date_to_watch(movie_id):
  print(f'getting last date to watch for movie {movie_id}')
  url = f'https://www.netflix.com/title/{movie_id}'
  # url = f'https://www.netflix.com/title/60020686'
  headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
    'Cookie': cookie
  }
  response = requests.get(url, headers=headers)

  if response.status_code != 200:
    raise Exception(f'Can not call API, status code: {response.status_code}, error: {response.text}')

  pattern = r'Last\\x20day\\x20to\\x20watch\\x20on\\x20Netflix:\\x20([A-Za-z]+)\\x20(\d+)'
  match = re.search(pattern, response.text)
  expire_date = None
  if match:
      month_name = match.group(1)
      day = int(match.group(2))
      month = datetime.datetime.strptime(month_name, "%B").month
      current_date_time = datetime.date.today()
      expire_date = current_date_time.replace(month=month, day=day)
  return expire_date

# loop through each row using iterrows() method
now = datetime.date.today()
for index, row in df.iterrows():
  # skip it has been processed
  last_fetch = df.at[index, 'Last Fetch']
  if isinstance(last_fetch, str) and (now - datetime.datetime.strptime(last_fetch, '%Y-%m-%d').date()).days < 5:
    continue
  
  # skip if is Netflix original
  if df.at[index, 'Netflix Original']:
    continue

  df.at[index, 'Last Day To Watch'] = get_last_date_to_watch(index)
  df.at[index, 'Last Fetch'] = now.isoformat()
df.head()

## Find movies to watch

In [ ]:
from IPython.display import display, HTML

df2 = df.copy()

# filter movies by genres
# df2 = df2[df2['Genres'].str.contains('Comedy')]

# watch expiring movies first, then by rating
df2 = df2[df2['Last Day To Watch'].notnull()] \
  .sort_values(by='Last Day To Watch', ascending=True, na_position='last')

# sort by rating descending
# df = df.sort_values(by='Imdb Rating', ascending=False)

# show movie list
df2['Netflix Link'] = df2.index
display(HTML(df2.to_html(
  columns=['Image', 'Title', 'Last Day To Watch', 'Imdb Rating', 'Movie Type', 'Netflix Link'],
  formatters={
    'Netflix Link': lambda val: f'<a href="https://www.netflix.com/title/{val}">{val}</a>',
    'Image': lambda val: f'<img src="{val}" width="171" height="96" />',
  },
  escape=False,
  index=False, 
)))